# Building a house (Harvard Business Review, 1963)

Several tasks must be completed in order to build our house. Each task has a duration, tasks may be worked on simultaneously, but there is also a precedence relation. Some tasks can only be started once other tasks have been completed. The following table shows each task, it's duration, and the tasks that must be completed before it starts. How fast can the house be built?
![alt text](https://hbr.org/resources/images/article_assets/hbr/6309/63508_A.gif)

## Problem Data

In [6]:
# this array stores the task names (:a, :b, ..., :x)
tasks = []
for i = 'a':'x'
    push!(tasks, Symbol(i))    # string(sym) converts back to a string, i.e. string(:hello) returns "hello"
end

# this dictionary stores the project durations
dur = [0, 4, 2, 4, 6, 1, 2, 3, 2, 4, 10, 3, 1, 2, 3, 2, 1, 1, 2, 3, 1, 2, 5, 0]
duration = Dict(zip(tasks,dur))

# this dictionary stores the projects that a given project depends on (ancestors)
pre = ( [], [:a], [:b], [:c], [:d], [:c], [:f], [:f], [:d], [:d,:g], [:i,:j,:h], [:k],
    [:l], [:l], [:l], [:e], [:p], [:c], [:o,:t], [:m,:n], [:t], [:q,:r], [:v], [:s,:u,:w])
pred = Dict(zip(tasks,pre));

In [7]:
pred

Dict{Any, Vector} with 24 entries:
  :o => [:l]
  :b => [:a]
  :p => [:e]
  :n => [:l]
  :j => [:d, :g]
  :e => [:d]
  :c => [:b]
  :h => [:f]
  :l => [:k]
  :w => [:v]
  :x => [:s, :u, :w]
  :d => [:c]
  :k => [:i, :j, :h]
  :s => [:o, :t]
  :v => [:q, :r]
  :g => [:f]
  :u => [:t]
  :q => [:p]
  :r => [:c]
  ⋮  => ⋮

## Mathematical Model

Decision variables:  
Start time of the task $t_i$ for all tasks $i \in \mathcal{T}$, where $\mathcal{T}$ is the set of all tasks.
Note that we define a finish task, which defines the last "task" and has duration 0.

Constraints:  
Each task can only start when the previous task is done,  
$$t_i \geq t_j + d_j\quad \forall j \in  \mathcal{P}_i \quad \forall i \in\mathcal{T}$$
where $\mathcal{P}_i$ is the set of all tasks that must be finished before task $i$ can start (i.e., predecessor tasks) and $d_j$ is the duration of task $j$.

The first task will start at time 0, i.e.,
$$t_a == 0$$

Objective:  
$$\min_{t} t_x + d_x$$
where $t_x$ and $d_x$ are the start time and duration of the finish task, respectively. 


## JuMP Implementation

In [8]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)

@variable(m, tstart[tasks])

# one-line implementation of the constraints:
# @constraint(m, link[i in tasks, j in pred[i]], tstart[i] >= tstart[j] + duration[j])

for i in tasks
    for j in pred[i]
        @constraint(m, tstart[i] >= tstart[j] + duration[j])
    end
end
@constraint(m, tstart[:a] == 0)
@objective(m, Min, tstart[:x] + duration[:x])     # total duration is start time of last task + duration of last task.   In this case, we know that :x is the last task, but in general, we must write this constraint for every task (or at least every task at the end of a chain in the parital order defined by the precedence)

optimize!(m)
println(value.(tstart))
println("minimum duration: ", objective_value(m))

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
30 rows, 22 cols, 59 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-32); columns 0(-24); elements 0(-63) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.4000000000e+01
HiGHS run time      :          0.00
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Any[:a, :b, :c, :d, :e, :f, :g, :h, :i, :j  …  :o, :p, :q, :r, :s, :t, :u, :v, :w, :x]
And data, a 24-element Vector{Float64}:
 -0.0
 -0.0
  4.0
  6.0
 10.0
  6.0
  7.0
  7.0
 10.0
 10.0
 14.0
 24.0
 27.0
 27.0
 27.0
 16.0
 18.0
  6.0
 32.0
 29.0
 32.0
 19.0
 21.0
 34.0
minimum duration: 34.0


In [9]:
for i in tasks
    for j in pred[i]
       slack = value(tstart[i]) - value(tstart[j]) - duration[j];
        if (slack <= .00001)
            println("task ", i, " predecessor ",j, "  slack is : ", slack," ***ACTIVE")
        else
             println("task ", i, " predecessor ",j, "  slack is : ", slack)
        end
    end
end

task b predecessor a  slack is : 0.0 ***ACTIVE
task c predecessor b  slack is : 0.0 ***ACTIVE
task d predecessor c  slack is : 0.0 ***ACTIVE
task e predecessor d  slack is : 0.0 ***ACTIVE
task f predecessor c  slack is : 0.0 ***ACTIVE
task g predecessor f  slack is : 0.0 ***ACTIVE
task h predecessor f  slack is : 0.0 ***ACTIVE
task i predecessor d  slack is : 0.0 ***ACTIVE
task j predecessor d  slack is : 0.0 ***ACTIVE
task j predecessor g  slack is : 1.0
task k predecessor i  slack is : 2.0
task k predecessor j  slack is : 0.0 ***ACTIVE
task k predecessor h  slack is : 4.0
task l predecessor k  slack is : 0.0 ***ACTIVE
task m predecessor l  slack is : 0.0 ***ACTIVE
task n predecessor l  slack is : 0.0 ***ACTIVE
task o predecessor l  slack is : 0.0 ***ACTIVE
task p predecessor e  slack is : 0.0 ***ACTIVE
task q predecessor p  slack is : 0.0 ***ACTIVE
task r predecessor c  slack is : 0.0 ***ACTIVE
task s predecessor o  slack is : 2.0
task s predecessor t  slack is : 0.0 ***ACTIVE
task t